In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install transformers
!pip install torch
!pip install numpy

In [11]:
import pandas as pd
from transformers import AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/data/fiction/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/data/fiction/test_x.csv')
submission = pd.read_csv('/content/drive/MyDrive/data/fiction/sample_submission.csv')

In [5]:
train_data.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [13]:
submission.head()

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0


In [ ]:
# GPU 사용 설정
device = 'cuda' if cuda.is_available() else 'cpu'

In [12]:
# 사전학습이 되지 않은 Bert모델을 불러옵니다.
from transformers import AutoConfig
config = AutoConfig.from_pretrained('bert-base-uncased')
bert =  AutoModel.from_config(config)

In [ ]:
class AuthorClassification(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes=5,
               dropout=None,
               params=None):
    super(AuthorClassification, self).__init__()
    self.bert = bert
    self.dropout = dropout
    self.classifier = nn.Linear(hiddden_size, num_classes)

  def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
      attention_mask = self.gen_attention_mask(token_ids, valid_length)
      
      _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
      if self.dr_rate:
          out = self.dropout(pooler)
      return self.classifier(out)

In [ ]:
model = AuthorClassification(bert, dropout=0.5).to(device)